# Steam Trike Calculations - Test
Like many design tasks, sizing the various components of the steam trike is an iterative process. We start with certain assumed parameters and goals, then develop cylinder and accumulator dimensions. These, in turn, affect piping and generator design. At various points in the design, we can compare the results against the initial assumptions and adjust.

In this notebook, we start with one set of assumed parameters as a demonstration, but develop the code in a way that we can evaluate different sets of parameters easily.

In [2]:
import numpy as np
!pip install pint
!pip install iapws
import pint
from iapws import IAPWS97 as steam
ureg = pint.UnitRegistry()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.0/294.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.9/111.9 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iapws: filename=iapws-1.5.3-py3-none-any.whl size=114269 sha256=110e2d17c15747b7bf3820c465ea5ddf3bddc7fa40b9925daba3058bd5424f99
  Stored in directory: /root/.cache/pip/wheels/4b/8f/d7/48fbfa9f897fcad5410a5c26a04ead1dfb131026e505bff24a
Successfully built iapws


## Initial Parameters

In [3]:
trike_weight = 2000 * ureg('lb')        # Based on homebuilt electric and CE trikes coming in at 800-1500 lb online
avg_pressure = 125 * ureg('lbf/in**2')  # Based on max of 150 psi saturated steam with margin
max_velocity = 55 * ureg('mi/hr')       # Based on a minimum speed for highway but not freeway travel

What's the maximum RPM we can expect to be able to achieve with the reciprocating engine? Common practice says 250-500 RPM, but we can check top RPMs of prototypical locomotives and steam cars.

| Prototype | Max Speed (mi/hr) | Driver Diameter (in) | rev/min |
| --- | --- | --- | --- |
| UP 844 | 120 | 80 | 504 |
| UP 4014 | 80 | 68 | 395 |
| Reading 2102 | 80 | 70 | 384 |
| PM 1225 | 70 | 69 | 341 |
| N&W 611 | 110 | 70 | 528 |
| SP 4449 | 110 | 80 | 462 |
| Stanley Steam Car | 60 | (N/A) | 900 |
| Doble Steam Car | 70 | (N/A) | 900 |

In [4]:
max_rpm = 350 * ureg('1/min')           # Conservative bet for larger reciprocating steam engine

## Required Wheel Diameter
From the max RPM and max velocity,

In [5]:
def wheel_dia():
    return max_velocity / max_rpm / np.pi
wheel_dia().to('in')

52.821251970155906 <Unit('inch')>

## Required Acceleration
Let's try two paths I have thought of to come up with a target maximum acceleration.

### Normal Acceleration Data
We want acceleration behavior compatible with traffic, so we can look at published data for conventional road vehicles. The data shown in Figures 1 and 2 are for consumer vehicles accelerating in normal traffic, not the maximum acceleration the vehicles are capable of. Thus, we want to target a value on the high end to reflect extra capacity available for "flooring it".


<img src="https://github.com/collinneedham/steam/blob/main/fig5e.PNG?raw=1" width="200"/><br />
Fig 1. Acceleration vs speed of various gasoline vehicles. Graph, "Scatter plots of acceleration-speed," from P.S.Bokare and A.K.Maurya, Acceleration-Deceleration Behaviour of Various Vehicle Types, page 10, figure 5(e).
<br />

<img src="https://github.com/collinneedham/steam/blob/main/fig6e.PNG?raw=1" width="200"/><br />
Fig 2. Idealized acceleration vs speed of gasoline vehicles. Graph, "Idealized plot of acceleration with speed for all vehicle types," from P.S.Bokare and A.K.Maurya, Acceleration-Deceleration Behaviour of Various Vehicle Types, page 11, figure 6(e).
<br />


The acceleration curves for these conventional vehicles include gear shifts and will not represent the curve for the steam trike. Instead, we are looking to have a comparable acceleration from a stop.

Candidate `max_accel` = 2 \[m/s^2\] ≅ 6.5 \[ft/s^2\]

### 0-60 MPH Time Data
0-60 MPH times for vehicles represent maximum acceleration, which is what we want. However, because car acceleration varies as the car accelerates, you're looking at an average max acceleration instead of the highest instantaneous acceleration. Data from ZeroTo60Times.com.

| Prototype | 0-60 MPH Time (s) | Acceleration (m/s^2) |
| --- | --- | --- |
| Geo Metro | 12.6 | 7.0 |
| Nissan Versa | 9.0 | 10.0 |
| Honda Fit | 7.8 | 11.3 |

### Selection
We see that the candidate max_accel from normal vehicle acceleration is lower than even the Geo Metro's 0-60 MPH, which makes sense as that was normal traffic acceleration instead of max. Given that we are looking for a usable max acceleration target, let's use that of the Geo Metro.

In [6]:
max_accel = 7.0 * ureg('ft/s**2')

## Cylinder Dimensions
### Required Force at Roadway
Required force on the road to accelerate `trike_weight` by `max_accel`

In [7]:
def road_force():
    return trike_weight * max_accel
road_force().to('lbf')

435.13330240194153 <Unit('force_pound')>

### Required Cylinder Volume
The mean tractive force of a two cylinder engine, at 90 percent cutoff, with the cylinders 90 degrees out of phase is given by:

F = K2 * K3 * avg_pressure * cyl_volume / wheel_dia
* K2 = 1.2 - The ratio of mean torque of two identical piston-crank assemblies 90 degrees out of phase to the peak torque of one of the piston-crank assemblies (Jeffrey Hook, Fundamentals of Steam Locomotive Tractive Force, page 7)
* K3 = 0.9 - A fudge factor to account for pressure loss between the boiler and the cylinder (Hook)
* cyl_volume - The swept volume of the cylinder

We will solve for the swept volume of the cylinder,

In [8]:
def cyl_volume():
    return road_force() * wheel_dia() * 1.2 * 0.9 / avg_pressure
cyl_volume().to('in**3')

198.58422937057057 <Unit('inch ** 3')>

### Required Cylinder Dimensions
We set cylinder bore as a parameter and calculate stroke,

In [9]:
cyl_bore = 5.0 * ureg('in')
def cyl_stroke():
    return cyl_volume() / (cyl_bore**2 * np.pi / 4)
cyl_stroke().to('in')

10.113811751814737 <Unit('inch')>

## Accumulator Sizing
### Maximum Steam Rate
Maximum steam rate occurs at max cutoff and maximum speed, and represents 4 strokes per revolution

In [10]:
def max_steam_vol_rate():
    return 0.9 * cyl_volume() * max_rpm * 4
max_steam_vol_rate().to('ft**3/min')

144.80100058270773 <Unit('foot ** 3 / minute')>

In [11]:

def max_steam_mass_rate():
    state = steam(P=avg_pressure.to('MPa').magnitude, x=1)
    v = state.v * ureg('m**3/kg')
    return max_steam_vol_rate().to('m**3/min') / v
max_steam_mass_rate().to('lb/min')


40.37114754565772 <Unit('pound / minute')>

In [12]:
def max_steam_liquid_rate():
    state = steam(P=0.1, T=300)
    v = state.v * ureg('m**3/kg')
    return max_steam_mass_rate() * v
max_steam_liquid_rate().to('gal/min')

4.854241187317671 <Unit('gallon / minute')>

Empirical data shows the relationship between available saturated water surface area, pressure, and the maximum steam release rate in an accumulator,

In [13]:

def accum_surface_area():
    area = max_steam_mass_rate().to('kg/hr').magnitude / 220 / (avg_pressure.to('bar').magnitude + 1)
    return (area * ureg('m**2'))
accum_surface_area().to('ft**2')

5.588954064921339 <Unit('foot ** 2')>